# Numerical Methods for Manu Body Physics, Assignment #3

Yoav Zack, ID 211677398

In [ ]:
using LinearAlgebra
using ITensors
using Statistics
using Plots
using LsqFit
using Printf
theme(:dracula)

ITensors.enable_debug_checks()

In [ ]:
using Base.Threads

const print_lock = SpinLock()
const prints_pending = Vector{String}()
function tprintln(str)
	tid= Threads.threadid()
	str = "[Thread $tid]: " * string(str)
	lock(print_lock) do
		push!(prints_pending, str)
		if tid == 1 # Only first thread is allows to print
			println.(prints_pending)
			empty!(prints_pending)
		end
	end
end

## Question 1: $J_1 - J_2$ Spin-1/2 Chain

We will use the `ITensors.jl` library to generate a Heisenberg Hamiltonian with general finite $J_1,J_2$ terms:

In [ ]:
function heisenbergH(sites, g::Real)
    N = length(sites)
    
    os = OpSum()
    for j in 1:N
        os += 1/2,"S+",j,"S-",mod(j,N)+1
        os += 1/2,"S-",j,"S+",mod(j,N)+1
        os += "Sz",j,"Sz",mod(j,N)+1
        
        os += 1/2*g,"S+",j,"S-",mod(mod(j,N)+1,N)+1
        os += 1/2*g,"S-",j,"S+",mod(mod(j,N)+1,N)+1
        os += g,"Sz",j,"Sz",mod(mod(j,N)+1,N)+1
    end
    
    H = MPO(os, sites)
    return H
end

We will use the DMRG method on this Hamiltonian to get the ground state and the 1st excited state. We will loop over system sizes, for each defining 
`sitetinds` object and the Hamiltonian, and for each finding the energy of the ground state with $S_z=0$ and 1st excited state with $S_z=1$:

In [ ]:
glist = [0.1, 0.2, 0.45, 0.48]
Nlist = 2 .^ (3:7)

nsweeps = 10
cutoff = [1E-8]
maxM = maximum(Nlist)

E0arr = zeros(length(glist), length(Nlist))
E1arr = zeros(length(glist), length(Nlist))

Threads.@threads for gind in 1:length(glist)
    g = glist[gind]
    for (Nind, N) in enumerate(Nlist)
        tprintln(string(g) * " " * string(N))
        J1 = 1.0
        J2 = g * J1
        
        sites = siteinds("S=1/2", N; conserve_qns=true)
        H = heisenbergH(sites, g);
    
        state = [isodd(n) ? "Up" : "Dn" for n=1:N]
        psi0 = MPS(sites,state);
        state[2] = "Up"
        psi1 = MPS(sites,state);
    
        E0arr[gind, Nind], psi_ground_1 = dmrg(H,psi0; nsweeps=nsweeps, cutoff=cutoff, maxdim=maxM, outputlevel=0);
        E1arr[gind, Nind], psi_ground_0 = dmrg(H,psi1; nsweeps=nsweeps, cutoff=cutoff, maxdim=maxM, outputlevel=0);
    end
end

In [ ]:
plot()
for (gind, g) in enumerate(glist)
    plot!(Nlist, E1arr[gind,:] .- E0arr[gind,:], label=@sprintf("g=%.2f",g), marker=:circle)
end
plot!(xlabel="System Size", ylabel="Energy Difference", xaxis=:log)

In [ ]:
tprintln(string(g) * " " * string(N))